In [1]:
# Ячейка 1: Импорты и настройка
import os
import pandas as pd
import polars as pl
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.feature_selection import VarianceThreshold
import warnings
import gc
from torch.utils.data import DataLoader, TensorDataset
from scipy.stats import spearmanr
import joblib
import optuna
from optuna.integration import PyTorchLightningPruningCallback
import plotly.express as px

warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используется устройство: {device}")
print(f"Доступно GPU: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"Название GPU: {torch.cuda.get_device_name(0)}")

NUM_TARGET_COLUMNS = 424

# Глобальные переменные
models = []
scaler = None
feature_selector = None
feature_cols = None
base_cols = None
is_initialized = False
model_val_scores = []
ensemble_weights_global = None
best_hyperparams = {}
optuna_studies = {}

Используется устройство: cuda
Доступно GPU: 1
Название GPU: NVIDIA L4


In [2]:
# Ячейка 2: Аугментация данных
class TimeSeriesAugmentation:
    """Продвинутая аугментация для финансовых временных рядов"""
    
    @staticmethod
    def add_gaussian_noise(features, noise_level=0.01):
        """Добавление гауссовского шума"""
        noise = torch.randn_like(features) * noise_level
        return features + noise
    
    @staticmethod
    def time_warp(features, warp_factor=0.1):
        """Временное искажение - сдвиг признаков во времени"""
        batch_size, seq_len = features.shape
        warp_strength = torch.randn(batch_size, 1, device=features.device) * warp_factor
        warped_indices = torch.arange(seq_len, device=features.device).float() + warp_strength
        warped_indices = torch.clamp(warped_indices, 0, seq_len-1)
        
        # Интерполяция
        warped_features = torch.zeros_like(features)
        for i in range(batch_size):
            warped_features[i] = torch.interp(
                torch.arange(seq_len, device=features.device).float(),
                warped_indices[i],
                features[i]
            )
        return warped_features
    
    @staticmethod
    def feature_dropout(features, drop_prob=0.1):
        """Случайное обнуление признаков (как dropout но для фичей)"""
        mask = torch.rand_like(features) > drop_prob
        return features * mask.float()
    
    @staticmethod
    def scaling_augmentation(features, scale_range=(0.9, 1.1)):
        """Случайное масштабирование признаков"""
        scale_factors = torch.rand(features.shape[0], 1, device=features.device) * (scale_range[1] - scale_range[0]) + scale_range[0]
        return features * scale_factors
    
    @staticmethod
    def jittering(features, jitter_strength=0.02):
        """Дрожание - случайные небольшие изменения"""
        jitter = (torch.rand_like(features) - 0.5) * 2 * jitter_strength
        return features + jitter
    
    @staticmethod
    def random_permutation(features, permute_fraction=0.1):
        """Случайная перестановка небольшой части признаков"""
        batch_size, num_features = features.shape
        num_to_permute = int(num_features * permute_fraction)
        
        for i in range(batch_size):
            perm_indices = torch.randperm(num_features)[:num_to_permute]
            features[i, perm_indices] = features[i, perm_indices[torch.randperm(num_to_permute)]]
        
        return features

def advanced_augmentation(features, augmentation_methods=None, augmentation_prob=0.7):
    """Продвинутая аугментация с комбинацией методов"""
    if augmentation_methods is None:
        augmentation_methods = [
            ('noise', 0.3),
            ('scaling', 0.2), 
            ('jitter', 0.2),
            ('dropout', 0.2),
            ('permutation', 0.1)
        ]
    
    augmented_features = features.clone()
    
    # Применяем аугментации с вероятностью
    if torch.rand(1).item() < augmentation_prob:
        for method, prob in augmentation_methods:
            if torch.rand(1).item() < prob:
                if method == 'noise':
                    augmented_features = TimeSeriesAugmentation.add_gaussian_noise(augmented_features, 0.01)
                elif method == 'scaling':
                    augmented_features = TimeSeriesAugmentation.scaling_augmentation(augmented_features)
                elif method == 'jitter':
                    augmented_features = TimeSeriesAugmentation.jittering(augmented_features)
                elif method == 'dropout':
                    augmented_features = TimeSeriesAugmentation.feature_dropout(augmented_features, 0.05)
                elif method == 'permutation':
                    augmented_features = TimeSeriesAugmentation.random_permutation(augmented_features, 0.05)
    
    return augmented_features

In [3]:
# Ячейка 3: Функции потерь
class PearsonCorrelationLoss(nn.Module):
    """Loss функция для оптимизации Pearson Correlation"""
    def __init__(self):
        super().__init__()
    
    def forward(self, pred, target):
        # Для каждого target отдельно
        batch_size = pred.shape[0]
        num_targets = pred.shape[1]
        
        total_loss = 0
        
        for i in range(num_targets):
            pred_col = pred[:, i]
            target_col = target[:, i]
            
            # Центрируем
            pred_centered = pred_col - pred_col.mean()
            target_centered = target_col - target_col.mean()
            
            # Ковариация
            covariance = (pred_centered * target_centered).mean()
            
            # Стандартные отклонения
            pred_std = pred_centered.std()
            target_std = target_centered.std()
            
            # Pearson correlation
            if pred_std > 1e-8 and target_std > 1e-8:
                correlation = covariance / (pred_std * target_std)
                # Минимизируем отрицательную корреляцию
                total_loss += -correlation
            else:
                # Штраф за константные предсказания
                total_loss += 1.0
        
        return total_loss / num_targets


class SpearmanLoss(nn.Module):
    """Approximation of Spearman correlation loss"""
    def __init__(self):
        super().__init__()
    
    def forward(self, pred, target):
        # Используем ранжирование через сортировку
        pred_rank = torch.argsort(torch.argsort(pred, dim=0), dim=0).float()
        target_rank = torch.argsort(torch.argsort(target, dim=0), dim=0).float()
        
        # Нормализуем ранги
        pred_rank = (pred_rank - pred_rank.mean(dim=0)) / (pred_rank.std(dim=0) + 1e-8)
        target_rank = (target_rank - target_rank.mean(dim=0)) / (target_rank.std(dim=0) + 1e-8)
        
        # Correlation (минимизируем отрицательную)
        correlation = (pred_rank * target_rank).mean(dim=0).mean()
        return -correlation


class CombinedCorrelationLoss(nn.Module):
    """Комбинация Pearson + Spearman + MSE"""
    def __init__(self, pearson_weight=0.4, spearman_weight=0.4, mse_weight=0.2):
        super().__init__()
        self.pearson_weight = pearson_weight
        self.spearman_weight = spearman_weight
        self.mse_weight = mse_weight
        
        self.pearson_loss = PearsonCorrelationLoss()
        self.spearman_loss = SpearmanLoss()
        self.mse_loss = nn.MSELoss()
    
    def forward(self, pred, target):
        pearson_l = self.pearson_loss(pred, target)
        spearman_l = self.spearman_loss(pred, target)
        mse_l = self.mse_loss(pred, target)
        
        total_loss = (self.pearson_weight * pearson_l + 
                     self.spearman_weight * spearman_l + 
                     self.mse_weight * mse_l)
        
        return total_loss


class SpearmanRankCorrelationLoss(nn.Module):
    """
    ПРАВИЛЬНАЯ Spearman Correlation Loss для Mitsui Competition
    
    Вычисляет корреляцию рангов ПО TARGETS внутри каждого sample
    (т.е. ранжирует 424 targets для каждого дня отдельно)
    """
    def __init__(self):
        super().__init__()
    
    def forward(self, pred, target):
        """
        pred, target: (batch_size, 424)
        
        Для каждого sample в батче:
          - Ранжируем 424 predictions
          - Ранжируем 424 true targets
          - Вычисляем Spearman correlation между этими рангами
        """
        batch_size = pred.shape[0]
        num_targets = pred.shape[1]
        
        correlations = []
        
        for i in range(batch_size):
            # Для i-го дня ранжируем все 424 targets
            pred_sample = pred[i]  # (424,)
            target_sample = target[i]  # (424,)
            
            # Получаем ранги (argsort дважды даёт ранги)
            pred_rank = pred_sample.argsort().argsort().float()
            target_rank = target_sample.argsort().argsort().float()
            
            # Нормализуем ранги
            pred_rank = (pred_rank - pred_rank.mean()) / (pred_rank.std() + 1e-8)
            target_rank = (target_rank - target_rank.mean()) / (target_rank.std() + 1e-8)
            
            # Pearson correlation на рангах = Spearman correlation
            correlation = (pred_rank * target_rank).mean()
            correlations.append(correlation)
        
        # Минимизируем отрицательную среднюю корреляцию
        mean_correlation = torch.stack(correlations).mean()
        
        return -mean_correlation


class DirectionalLoss(nn.Module):
    """
    Штрафует за неправильное направление (знак)
    """
    def __init__(self):
        super().__init__()
    
    def forward(self, pred, target):
        # Совпадает ли знак?
        pred_sign = torch.sign(pred)
        target_sign = torch.sign(target)
        direction_correct = (pred_sign == target_sign).float()
        
        # Штрафуем за несовпадение
        direction_loss = 1 - direction_correct.mean()
        
        return direction_loss


class MitsuiOptimizedLoss(nn.Module):
    """
    ОПТИМИЗИРОВАННАЯ Loss функция для Mitsui Competition
    
    Комбинирует:
    - Spearman Rank Correlation (главное!)
    - MSE (для magnitude)
    - Directional Loss (для знака)
    """
    def __init__(self, spearman_weight=0.6, mse_weight=0.2, direction_weight=0.2):
        super().__init__()
        self.spearman_weight = spearman_weight
        self.mse_weight = mse_weight
        self.direction_weight = direction_weight
        
        self.spearman_loss = SpearmanRankCorrelationLoss()
        self.directional_loss = DirectionalLoss()
        self.mse_loss = nn.MSELoss()
    
    def forward(self, pred, target):
        # Три компоненты
        spearman_l = self.spearman_loss(pred, target)
        mse_l = self.mse_loss(pred, target)
        direction_l = self.directional_loss(pred, target)
        
        # Взвешенная сумма
        total_loss = (
            self.spearman_weight * spearman_l +
            self.mse_weight * mse_l +
            self.direction_weight * direction_l
        )
        
        return total_loss

In [4]:
# Ячейка 4: Архитектуры моделей с поддержкой гиперпараметров
class SimpleEffectiveModel_Improved(nn.Module):
    def __init__(self, input_size, output_size, hidden_size=512, dropout_rate=0.4, num_layers=4):
        super().__init__()
        
        layers = []
        current_size = input_size
        
        for i in range(num_layers):
            next_size = hidden_size // (2 ** i) if i < num_layers - 1 else output_size
            layers.extend([
                nn.Linear(current_size, next_size),
                nn.BatchNorm1d(next_size) if i < num_layers - 1 else nn.Identity(),
                nn.ReLU() if i < num_layers - 1 else nn.Identity(),
                nn.Dropout(dropout_rate * (0.8 ** i)) if i < num_layers - 1 else nn.Identity()
            ])
            current_size = next_size
        
        self.net = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.net(x)

class ResidualModel(nn.Module):
    """Модель с residual connections"""
    def __init__(self, input_size, output_size, hidden_size=256, num_blocks=2, dropout_rate=0.2):
        super().__init__()
        
        self.input_proj = nn.Linear(input_size, hidden_size)
        
        self.blocks = nn.ModuleList()
        for _ in range(num_blocks):
            self.blocks.append(nn.Sequential(
                nn.Linear(hidden_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate),
                nn.Linear(hidden_size, hidden_size),
                nn.BatchNorm1d(hidden_size)
            ))
        
        self.output = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(dropout_rate * 0.5),
            nn.Linear(hidden_size // 2, output_size)
        )
        
        self.dropout = nn.Dropout(dropout_rate)
    
    def forward(self, x):
        x = self.input_proj(x)
        
        # Residual blocks
        for block in self.blocks:
            residual = x
            x = block(x)
            x = x + residual
            x = nn.ReLU()(x)
            x = self.dropout(x)
        
        return self.output(x)

class WideModel(nn.Module):
    """Широкая модель с большим количеством нейронов"""
    def __init__(self, input_size, output_size, hidden_sizes=[1024, 512, 256], dropout_rates=[0.4, 0.3, 0.2]):
        super().__init__()
        
        layers = []
        current_size = input_size
        
        for i, (hidden_size, dropout_rate) in enumerate(zip(hidden_sizes, dropout_rates)):
            layers.extend([
                nn.Linear(current_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            current_size = hidden_size
        
        layers.append(nn.Linear(current_size, output_size))
        
        self.net = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.net(x)

class DeepSkipModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_size=512, num_blocks=4, dropout_rate=0.3):
        super().__init__()
        
        self.input_proj = nn.Linear(input_size, hidden_size)
        
        self.blocks = nn.ModuleList()
        for i in range(num_blocks):
            self.blocks.append(nn.Sequential(
                nn.Linear(hidden_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate),
                nn.Linear(hidden_size, hidden_size),
                nn.BatchNorm1d(hidden_size)
            ))
        
        self.output = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_rate * 0.7)
    
    def forward(self, x):
        x = self.input_proj(x)
        
        for block in self.blocks:
            residual = x
            x = block(x)
            x = x + residual
            x = nn.ReLU()(x)
            x = self.dropout(x)
        
        return self.output(x)

class AttentionModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_size=512, num_heads=8, dropout_rate=0.1):
        super().__init__()
        
        self.input_proj = nn.Linear(input_size, hidden_size)
        
        # Self-attention
        self.attention = nn.MultiheadAttention(
            embed_dim=hidden_size,
            num_heads=num_heads,
            dropout=dropout_rate,
            batch_first=True
        )
        
        self.ffn = nn.Sequential(
            nn.Linear(hidden_size, hidden_size * 2),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size * 2, hidden_size)
        )
        
        self.norm1 = nn.LayerNorm(hidden_size)
        self.norm2 = nn.LayerNorm(hidden_size)
        self.output = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_rate)
    
    def forward(self, x):
        x = self.input_proj(x)
        
        # Add sequence dimension for attention
        x_seq = x.unsqueeze(1)
        
        # Self-attention with residual
        attn_out, _ = self.attention(x_seq, x_seq, x_seq)
        x = x + attn_out.squeeze(1)
        x = self.norm1(x)
        x = self.dropout(x)
        
        # Feed-forward with residual
        ffn_out = self.ffn(x)
        x = x + ffn_out
        x = self.norm2(x)
        x = self.dropout(x)
        
        return self.output(x)

class EnsembleInsideModel(nn.Module):
    def __init__(self, input_size, output_size, num_experts=3, expert_hidden=384, gate_hidden=128):
        super().__init__()
        
        self.experts = nn.ModuleList([
            nn.Sequential(
                nn.Linear(input_size, expert_hidden),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(expert_hidden, expert_hidden // 2)
            ) for _ in range(num_experts)
        ])
        
        self.gate = nn.Sequential(
            nn.Linear(input_size, gate_hidden),
            nn.ReLU(),
            nn.Linear(gate_hidden, num_experts),
            nn.Softmax(dim=1)
        )
        
        self.output = nn.Sequential(
            nn.Linear(expert_hidden // 2, gate_hidden),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(gate_hidden, output_size)
        )
    
    def forward(self, x):
        gates = self.gate(x)
        expert_outputs = [expert(x) for expert in self.experts]
        
        # Weighted combination
        combined = torch.zeros_like(expert_outputs[0])
        for i, expert_out in enumerate(expert_outputs):
            combined += gates[:, i:i+1] * expert_out
        
        return self.output(combined)

In [5]:
# Ячейка 5: Создание признаков
def create_optimized_features(df, base_cols_ref=None, max_features=800):
    """Создает оптимизированный набор признаков"""
    features = df.copy()
    
    if base_cols_ref is not None:
        numeric_cols = [c for c in base_cols_ref if c in df.columns]
    else:
        numeric_cols = [c for c in df.columns 
                       if c not in ['date_id', 'is_scored'] 
                       and pd.api.types.is_numeric_dtype(df[c])]
    
    # Ограничиваем количество базовых колонок
    numeric_cols = numeric_cols[:min(100, len(numeric_cols))]
    
    print(f"Обрабатывается {len(numeric_cols)} базовых колонок...")
    
    feature_count = 0
    for col in numeric_cols:
        try:
            with np.errstate(divide='ignore', invalid='ignore'):
                # Базовые returns
                features[f'{col}_return_1d'] = df[col].pct_change(1)
                features[f'{col}_return_5d'] = df[col].pct_change(5)
                
                # Скользящие статистики
                for window in [5, 10, 20]:
                    features[f'{col}_ma_{window}'] = df[col].rolling(window, min_periods=1).mean()
                    features[f'{col}_std_{window}'] = df[col].rolling(window, min_periods=1).std()
                
                # Z-score
                rolling_mean = df[col].rolling(20, min_periods=1).mean()
                rolling_std = df[col].rolling(20, min_periods=1).std()
                features[f'{col}_zscore'] = (df[col] - rolling_mean) / (rolling_std + 1e-8)
                
                # Lag features
                for lag in [1, 2, 3]:
                    features[f'{col}_lag_{lag}'] = df[col].shift(lag)
                
                # Volatility
                features[f'{col}_volatility_20'] = df[col].rolling(20).std() / (df[col].rolling(20).mean() + 1e-8)
                
                feature_count += 11  # 11 features per column
                
                # Останавливаемся если достигли лимита
                if feature_count >= max_features:
                    print(f"Достигнут лимит в {max_features} признаков")
                    break
                    
        except Exception as e:
            continue
    
    # Заполняем NaN
    features = features.fillna(0)
    
    # Удаляем колонки с нулевой дисперсией
    selector = VarianceThreshold()
    non_constant_features = selector.fit(features.select_dtypes(include=[np.number])).get_support()
    feature_columns = features.select_dtypes(include=[np.number]).columns[non_constant_features]
    features = features[feature_columns]
    
    print(f"Создано {len(features.columns)} признаков после фильтрации")
    
    return features

def select_best_features_vectorized_fixed(features, targets, n_features=500):
    """Исправленная векторизованная версия с обработкой NaN"""
    import torch
    
    print("Векторизованный отбор признаков с GPU...")
    
    # Фильтруем колонки
    feature_cols = [c for c in features.columns if c not in ['date_id', 'is_scored']]
    features_filtered = features[feature_cols]
    
    # Перенос на GPU
    X = torch.tensor(features_filtered.values, dtype=torch.float32, device=device)
    y = torch.tensor(targets.values, dtype=torch.float32, device=device)
    
    # Заменяем NaN и Inf на 0
    X = torch.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
    y = torch.nan_to_num(y, nan=0.0, posinf=0.0, neginf=0.0)
    
    # Центрируем данные
    X_centered = X - X.mean(dim=0)
    y_centered = y - y.mean(dim=0)
    
    # Вычисляем ковариационную матрицу
    covariance = torch.mm(X_centered.T, y_centered) / (X.shape[0] - 1)
    
    # Вычисляем стандартные отклонения с защитой от деления на 0
    x_std = X_centered.std(dim=0, unbiased=True)
    y_std = y_centered.std(dim=0, unbiased=True)
    
    # Защита от нулевых стандартных отклонений
    x_std = torch.where(x_std < 1e-8, torch.ones_like(x_std), x_std)
    y_std = torch.where(y_std < 1e-8, torch.ones_like(y_std), y_std)
    
    # Вычисляем матрицу корреляций Пирсона
    correlation_matrix = covariance / (x_std.unsqueeze(1) * y_std.unsqueeze(0))
    
    # Заменяем NaN в корреляциях на 0
    correlation_matrix = torch.nan_to_num(correlation_matrix, nan=0.0)
    
    # Усредняем абсолютные корреляции по targets
    feature_importances = correlation_matrix.abs().mean(dim=1)
    
    # Сортируем признаки по важности
    sorted_indices = torch.argsort(feature_importances, descending=True)
    selected_features = [feature_cols[i] for i in sorted_indices[:n_features].cpu().numpy()]
    
    print(f"Отобрано {len(selected_features)} лучших признаков")
    if len(feature_importances) > 0:
        best_score = feature_importances[sorted_indices[0]].item()
        print(f"Лучший признак: {selected_features[0]} (корреляция: {best_score:.4f})")
    
    # Очистка памяти
    del X, y, X_centered, y_centered, covariance, correlation_matrix
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    return selected_features

In [6]:
# Ячейка 6: Функции обучения с поддержкой Optuna
def calculate_spearman_score(predictions, targets):
    """Вычисляет средний Spearman correlation"""
    correlations = []
    
    for i in range(targets.shape[1]):
        pred_col = predictions[:, i]
        target_col = targets[:, i]
        
        if len(np.unique(pred_col)) > 1 and len(np.unique(target_col)) > 1:
            try:
                corr, _ = spearmanr(pred_col, target_col)
                if not np.isnan(corr):
                    correlations.append(corr)
            except:
                continue
    
    return np.mean(correlations) if correlations else 0.0

def train_model_with_augmentation(model, X_train_t, y_train_t, X_val_t, y_val_t, 
                                epochs=500, patience=50, lr=0.001, model_name="", 
                                use_augmentation=True, augmentation_strength=0.7,
                                trial=None):
    """Функция обучения с продвинутой аугментацией и поддержкой Optuna"""
    
    # criterion = CombinedCorrelationLoss()
    criterion = MitsuiOptimizedLoss(
        spearman_weight=0.6,
        mse_weight=0.2,
        direction_weight=0.2
    )
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.05)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=8, factor=0.5)
    
    best_val_score = -float('inf')
    best_model_state = None
    epochs_no_improve = 0
    
    if trial is None:
        print(f"Начало обучения {model_name} с аугментацией: {use_augmentation}")
    
    model.train()
    
    for epoch in range(epochs):
        if epochs_no_improve >= patience and trial is None:
            if trial is None:
                print(f"Early stopping на эпохе {epoch+1}")
            break
        
        # Обучение с аугментацией
        optimizer.zero_grad()
        
        if use_augmentation:
            # Применяем аугментацию только к тренировочным данным
            augmented_features = advanced_augmentation(X_train_t, augmentation_prob=augmentation_strength)
            train_outputs = model(augmented_features)
        else:
            train_outputs = model(X_train_t)
            
        train_loss = criterion(train_outputs, y_train_t)
        train_loss.backward()
        
        # Gradient clipping для стабильности
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        # Валидация (БЕЗ аугментации)
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_t)
            val_score = calculate_spearman_score(val_outputs.cpu().numpy(), y_val_t.cpu().numpy())
            
            train_preds = train_outputs.detach().cpu().numpy()
            train_true = y_train_t.cpu().numpy()
            train_score = calculate_spearman_score(train_preds, train_true)
        
        # Для Optuna: сообщаем промежуточное значение
        if trial is not None:
            trial.report(val_score, epoch)
            
            # Проверяем, нужно ли прервать trial
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()
        
        # Обновление лучших результатов
        if val_score > best_val_score:
            best_val_score = val_score
            best_model_state = model.state_dict().copy()
            epochs_no_improve = 0
            improvement_msg = "✓ УЛУЧШЕНИЕ"
        else:
            epochs_no_improve += 1
            improvement_msg = f"NO IMPROVE ({epochs_no_improve}/{patience})"
        
        model.train()
        scheduler.step(train_loss)
        
        # Логирование (только для обычного обучения, не для Optuna)
        if trial is None and ((epoch + 1) % 10 == 0 or epoch == 0 or epochs_no_improve >= patience):
            current_lr = optimizer.param_groups[0]['lr']
            aug_status = "AUG" if use_augmentation else "NO_AUG"
            print(f"{model_name} | {aug_status} | Ep {epoch+1:3d}/{epochs} | "
                  f"TrL: {train_loss.item():.6f} | "
                  f"TrScore: {train_score:.4f} | "
                  f"ValScore: {val_score:.4f} | "
                  f"LR: {current_lr:.6f} | {improvement_msg}")
    
    # Загрузка лучших весов
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    model.eval()
    
    return model, best_val_score

In [7]:
# Ячейка 7: Функции Optuna для оптимизации гиперпараметров
def objective_simple_effective(trial, X_train_t, y_train_t, X_val_t, y_val_t):
    """Objective функция для SimpleEffectiveModel"""
    
    # Предлагаем гиперпараметры
    hidden_size = trial.suggest_categorical('hidden_size', [256, 512, 1024])
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    num_layers = trial.suggest_int('num_layers', 3, 6)
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    augmentation_strength = trial.suggest_float('augmentation_strength', 0.3, 0.9)
    
    # Создаем модель
    model = SimpleEffectiveModel_Improved(
        input_size=X_train_t.shape[1],
        output_size=424,
        hidden_size=hidden_size,
        dropout_rate=dropout_rate,
        num_layers=num_layers
    ).to(device)
    
    # Обучаем модель
    try:
        model, val_score = train_model_with_augmentation(
            model, X_train_t, y_train_t, X_val_t, y_val_t,
            epochs=100,  # Меньше эпох для быстрой оптимизации
            patience=15,
            lr=lr,
            model_name=f"Optuna_SimpleEffective_{trial.number}",
            use_augmentation=True,
            augmentation_strength=augmentation_strength,
            trial=trial
        )
        
        return val_score
    
    except Exception as e:
        print(f"Ошибка в trial {trial.number}: {e}")
        return 0.0

def objective_residual(trial, X_train_t, y_train_t, X_val_t, y_val_t):
    """Objective функция для ResidualModel"""
    
    hidden_size = trial.suggest_categorical('hidden_size', [128, 256, 512])
    num_blocks = trial.suggest_int('num_blocks', 2, 5)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.4)
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    
    model = ResidualModel(
        input_size=X_train_t.shape[1],
        output_size=424,
        hidden_size=hidden_size,
        num_blocks=num_blocks,
        dropout_rate=dropout_rate
    ).to(device)
    
    try:
        model, val_score = train_model_with_augmentation(
            model, X_train_t, y_train_t, X_val_t, y_val_t,
            epochs=100,
            patience=15,
            lr=lr,
            model_name=f"Optuna_Residual_{trial.number}",
            use_augmentation=True,
            augmentation_strength=0.7,
            trial=trial
        )
        
        return val_score
    
    except Exception as e:
        print(f"Ошибка в trial {trial.number}: {e}")
        return 0.0

def objective_wide(trial, X_train_t, y_train_t, X_val_t, y_val_t):
    """Objective функция для WideModel"""
    
    hidden_size1 = trial.suggest_categorical('hidden_size1', [512, 1024, 2048])
    hidden_size2 = trial.suggest_categorical('hidden_size2', [256, 512, 1024])
    hidden_size3 = trial.suggest_categorical('hidden_size3', [128, 256, 512])
    dropout1 = trial.suggest_float('dropout1', 0.2, 0.6)
    dropout2 = trial.suggest_float('dropout2', 0.1, 0.5)
    dropout3 = trial.suggest_float('dropout3', 0.1, 0.4)
    lr = trial.suggest_float('lr', 1e-5, 1e-3, log=True)
    
    model = WideModel(
        input_size=X_train_t.shape[1],
        output_size=424,
        hidden_sizes=[hidden_size1, hidden_size2, hidden_size3],
        dropout_rates=[dropout1, dropout2, dropout3]
    ).to(device)
    
    try:
        model, val_score = train_model_with_augmentation(
            model, X_train_t, y_train_t, X_val_t, y_val_t,
            epochs=100,
            patience=15,
            lr=lr,
            model_name=f"Optuna_Wide_{trial.number}",
            use_augmentation=True,
            augmentation_strength=0.6,
            trial=trial
        )
        
        return val_score
    
    except Exception as e:
        print(f"Ошибка в trial {trial.number}: {e}")
        return 0.0

def optimize_model_with_optuna(model_type, X_train_t, y_train_t, X_val_t, y_val_t, n_trials=50):
    """Оптимизация гиперпараметров модели с помощью Optuna"""
    
    print(f"\n{'='*70}")
    print(f"OPTUNA ОПТИМИЗАЦИЯ ДЛЯ {model_type}")
    print(f"Количество trials: {n_trials}")
    print(f"{'='*70}")
    
    # Выбираем соответствующую objective функцию
    if model_type == "SimpleEffective":
        objective_func = lambda trial: objective_simple_effective(trial, X_train_t, y_train_t, X_val_t, y_val_t)
    elif model_type == "Residual":
        objective_func = lambda trial: objective_residual(trial, X_train_t, y_train_t, X_val_t, y_val_t)
    elif model_type == "Wide":
        objective_func = lambda trial: objective_wide(trial, X_train_t, y_train_t, X_val_t, y_val_t)
    elif model_type == "DeepSkip":
        objective_func = lambda trial: objective_deep_skip(trial, X_train_t, y_train_t, X_val_t, y_val_t)
    elif model_type == "Attention":
        objective_func = lambda trial: objective_attention(trial, X_train_t, y_train_t, X_val_t, y_val_t)
    elif model_type == "EnsembleInside":
        objective_func = lambda trial: objective_ensemble_inside(trial, X_train_t, y_train_t, X_val_t, y_val_t)
    else:
        raise ValueError(f"Неизвестный тип модели: {model_type}")
    
    # Создаем study
    study = optuna.create_study(
        direction='maximize',
        pruner=optuna.pruners.HyperbandPruner(
            min_resource=10,
            max_resource=100,
            reduction_factor=3
        )
    )
    
    # Запускаем оптимизацию
    study.optimize(objective_func, n_trials=n_trials, show_progress_bar=True)
    
    # Сохраняем результаты
    optuna_studies[model_type] = study
    best_hyperparams[model_type] = study.best_trial.params
    
    print(f"\nЛУЧШИЕ ГИПЕРПАРАМЕТРЫ ДЛЯ {model_type}:")
    for key, value in study.best_trial.params.items():
        print(f"  {key}: {value}")
    print(f"Лучший val_score: {study.best_value:.4f}")
    
    return study.best_trial.params, study.best_value

# Добавить в Ячейку 7: Функции Optuna для новых моделей

def objective_deep_skip(trial, X_train_t, y_train_t, X_val_t, y_val_t):
    """Objective функция для DeepSkipModel"""
    
    hidden_size = trial.suggest_categorical('hidden_size', [256, 512, 1024])
    num_blocks = trial.suggest_int('num_blocks', 2, 6)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    augmentation_strength = trial.suggest_float('augmentation_strength', 0.3, 0.9)
    
    model = DeepSkipModel(
        input_size=X_train_t.shape[1],
        output_size=424,
        hidden_size=hidden_size,
        num_blocks=num_blocks,
        dropout_rate=dropout_rate
    ).to(device)
    
    try:
        model, val_score = train_model_with_augmentation(
            model, X_train_t, y_train_t, X_val_t, y_val_t,
            epochs=100,
            patience=15,
            lr=lr,
            model_name=f"Optuna_DeepSkip_{trial.number}",
            use_augmentation=True,
            augmentation_strength=augmentation_strength,
            trial=trial
        )
        
        return val_score
    
    except Exception as e:
        print(f"Ошибка в trial {trial.number}: {e}")
        return 0.0

def objective_attention(trial, X_train_t, y_train_t, X_val_t, y_val_t):
    """Objective функция для AttentionModel"""
    
    hidden_size = trial.suggest_categorical('hidden_size', [256, 512, 1024])
    num_heads = trial.suggest_categorical('num_heads', [4, 8, 16])
    dropout_rate = trial.suggest_float('dropout_rate', 0.05, 0.3)
    lr = trial.suggest_float('lr', 1e-5, 1e-3, log=True)
    augmentation_strength = trial.suggest_float('augmentation_strength', 0.3, 0.7)
    
    model = AttentionModel(
        input_size=X_train_t.shape[1],
        output_size=424,
        hidden_size=hidden_size,
        num_heads=num_heads,
        dropout_rate=dropout_rate
    ).to(device)
    
    try:
        model, val_score = train_model_with_augmentation(
            model, X_train_t, y_train_t, X_val_t, y_val_t,
            epochs=100,
            patience=15,
            lr=lr,
            model_name=f"Optuna_Attention_{trial.number}",
            use_augmentation=True,
            augmentation_strength=augmentation_strength,
            trial=trial
        )
        
        return val_score
    
    except Exception as e:
        print(f"Ошибка в trial {trial.number}: {e}")
        return 0.0

def objective_ensemble_inside(trial, X_train_t, y_train_t, X_val_t, y_val_t):
    """Objective функция для EnsembleInsideModel"""
    
    num_experts = trial.suggest_int('num_experts', 2, 5)
    expert_hidden = trial.suggest_categorical('expert_hidden', [256, 384, 512])
    gate_hidden = trial.suggest_categorical('gate_hidden', [64, 128, 256])
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    augmentation_strength = trial.suggest_float('augmentation_strength', 0.3, 0.9)
    
    model = EnsembleInsideModel(
        input_size=X_train_t.shape[1],
        output_size=424,
        num_experts=num_experts,
        expert_hidden=expert_hidden,
        gate_hidden=gate_hidden
    ).to(device)
    
    try:
        model, val_score = train_model_with_augmentation(
            model, X_train_t, y_train_t, X_val_t, y_val_t,
            epochs=100,
            patience=15,
            lr=lr,
            model_name=f"Optuna_EnsembleInside_{trial.number}",
            use_augmentation=True,
            augmentation_strength=augmentation_strength,
            trial=trial
        )
        
        return val_score
    
    except Exception as e:
        print(f"Ошибка в trial {trial.number}: {e}")
        return 0.0

In [8]:
# Ячейка 8: Инициализация моделей с Optuna (ИСПРАВЛЕННАЯ)
def initialize_models_with_optuna(use_optuna=True, optuna_trials=30):
    global models, scaler, feature_cols, base_cols, model_val_scores, is_initialized, device
    global best_hyperparams, optuna_studies
    
    if is_initialized:
        return
    
    print("="*70)
    print(f"ИНИЦИАЛИЗАЦИЯ ВСЕХ 6 МОДЕЛЕЙ С OPTUNA ОПТИМИЗАЦИЕЙ НА {device}")
    print("="*70)
    
    # Загрузка данных
    train = pd.read_csv('/home/nicolaedrabcinski/sd_kaggle/data/raw/train.csv')
    train_labels = pd.read_csv('/home/nicolaedrabcinski/sd_kaggle/data/raw/train_labels.csv')
    
    # Базовые колонки
    base_cols = [c for c in train.columns 
                 if c not in ['date_id'] 
                 and pd.api.types.is_numeric_dtype(train[c])]
    
    print(f"Базовых колонок: {len(base_cols)}")
    print("Создание оптимизированных признаков...")
    
    # Создание признаков
    train_features = create_optimized_features(train, base_cols_ref=base_cols, max_features=800)
    
    # Выбор лучших признаков
    target_cols = [f'target_{i}' for i in range(424)]
    selected_features = select_best_features_vectorized_fixed(train_features, train_labels[target_cols], n_features=500)
    
    feature_cols = selected_features
    
    # Подготовка данных
    X_train = train_features[feature_cols].fillna(0).values
    y_train = train_labels[target_cols].fillna(0).values
    
    X_train = np.nan_to_num(X_train, nan=0.0, posinf=0.0, neginf=0.0)
    y_train = np.nan_to_num(y_train, nan=0.0, posinf=0.0, neginf=0.0)
    
    # Масштабирование
    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    
    # Перенос на GPU
    X_tensor = torch.FloatTensor(X_train_scaled).to(device)
    y_tensor = torch.FloatTensor(y_train).to(device)
    
    # Разделение на train/validation
    split_idx = int(len(X_train_scaled) * 0.85)
    X_train_t, X_val_t = X_tensor[:split_idx], X_tensor[split_idx:]
    y_train_t, y_val_t = y_tensor[:split_idx], y_tensor[split_idx:]
    
    print(f"Train: {len(X_train_t)}, Validation: {len(X_val_t)}")
    
    if use_optuna:
        # Оптимизация гиперпараметров для ВСЕХ 6 моделей
        models_to_optimize = ["SimpleEffective", "Residual", "Wide", "DeepSkip", "Attention", "EnsembleInside"]
        
        for model_type in models_to_optimize:
            try:
                best_params, best_score = optimize_model_with_optuna(
                    model_type, X_train_t, y_train_t, X_val_t, y_val_t, n_trials=optuna_trials
                )
                print(f"✅ {model_type} оптимизирована. Best score: {best_score:.4f}")
            except Exception as e:
                print(f"❌ Ошибка оптимизации {model_type}: {e}")
                continue
    
    # Конфигурация ВСЕХ 6 моделей с оптимизированными гиперпараметрами
    model_configs = []
    
    # SimpleEffective с оптимизированными параметрами
    if "SimpleEffective" in best_hyperparams:
        params = best_hyperparams["SimpleEffective"]
        model_configs.append(
            (SimpleEffectiveModel_Improved, "SimpleEffective_Optuna", params.get('lr', 0.001), 
             True, params.get('augmentation_strength', 0.7), params)
        )
    else:
        model_configs.append(
            (SimpleEffectiveModel_Improved, "SimpleEffective_Default", 0.001, True, 0.7, {})
        )
    
    # Residual с оптимизированными параметрами
    if "Residual" in best_hyperparams:
        params = best_hyperparams["Residual"]
        model_configs.append(
            (ResidualModel, "Residual_Optuna", params.get('lr', 0.001), 
             True, 0.7, params)
        )
    else:
        model_configs.append(
            (ResidualModel, "Residual_Default", 0.001, True, 0.7, {})
        )
    
    # Wide с оптимизированными параметрами
    if "Wide" in best_hyperparams:
        params = best_hyperparams["Wide"]
        model_configs.append(
            (WideModel, "Wide_Optuna", params.get('lr', 0.0005), 
             True, 0.6, params)
        )
    else:
        model_configs.append(
            (WideModel, "Wide_Default", 0.0005, True, 0.6, {})
        )
    
    # DeepSkip с оптимизированными параметрами
    if "DeepSkip" in best_hyperparams:
        params = best_hyperparams["DeepSkip"]
        model_configs.append(
            (DeepSkipModel, "DeepSkip_Optuna", params.get('lr', 0.001), 
             True, 0.7, params)
        )
    else:
        model_configs.append(
            (DeepSkipModel, "DeepSkip_Default", 0.001, True, 0.7, {})
        )
    
    # Attention с оптимизированными параметрами
    if "Attention" in best_hyperparams:
        params = best_hyperparams["Attention"]
        model_configs.append(
            (AttentionModel, "Attention_Optuna", params.get('lr', 0.0008), 
             True, 0.5, params)
        )
    else:
        model_configs.append(
            (AttentionModel, "Attention_Default", 0.0008, True, 0.5, {})
        )
    
    # EnsembleInside с оптимизированными параметрами
    if "EnsembleInside" in best_hyperparams:
        params = best_hyperparams["EnsembleInside"]
        model_configs.append(
            (EnsembleInsideModel, "EnsembleInside_Optuna", params.get('lr', 0.001), 
             True, 0.7, params)
        )
    else:
        model_configs.append(
            (EnsembleInsideModel, "EnsembleInside_Default", 0.001, True, 0.7, {})
        )
    
    # Обучение всех 6 моделей
    for i, (ModelClass, name, lr, use_aug, aug_strength, hyperparams) in enumerate(model_configs):
        print(f"\n{'='*70}")
        print(f"МОДЕЛЬ {i+1}/{len(model_configs)}: {name}")
        print(f"{'='*70}")
        
        try:
            # Создание модели с гиперпараметрами
            if "Optuna" in name:
                print("Используются оптимизированные гиперпараметры:")
                for key, value in hyperparams.items():
                    print(f"  {key}: {value}")
            
            if name == "SimpleEffective_Optuna":
                model = ModelClass(
                    X_train_scaled.shape[1], 424,
                    hidden_size=hyperparams.get('hidden_size', 512),
                    dropout_rate=hyperparams.get('dropout_rate', 0.4),
                    num_layers=hyperparams.get('num_layers', 4)
                ).to(device)
            elif name == "Residual_Optuna":
                model = ModelClass(
                    X_train_scaled.shape[1], 424,
                    hidden_size=hyperparams.get('hidden_size', 256),
                    num_blocks=hyperparams.get('num_blocks', 2),
                    dropout_rate=hyperparams.get('dropout_rate', 0.2)
                ).to(device)
            elif name == "Wide_Optuna":
                model = ModelClass(
                    X_train_scaled.shape[1], 424,
                    hidden_sizes=[
                        hyperparams.get('hidden_size1', 1024),
                        hyperparams.get('hidden_size2', 512),
                        hyperparams.get('hidden_size3', 256)
                    ],
                    dropout_rates=[
                        hyperparams.get('dropout1', 0.4),
                        hyperparams.get('dropout2', 0.3),
                        hyperparams.get('dropout3', 0.2)
                    ]
                ).to(device)
            elif name == "DeepSkip_Optuna":
                model = ModelClass(
                    X_train_scaled.shape[1], 424,
                    hidden_size=hyperparams.get('hidden_size', 512),
                    num_blocks=hyperparams.get('num_blocks', 4),
                    dropout_rate=hyperparams.get('dropout_rate', 0.3)
                ).to(device)
            elif name == "Attention_Optuna":
                model = ModelClass(
                    X_train_scaled.shape[1], 424,
                    hidden_size=hyperparams.get('hidden_size', 512),
                    num_heads=hyperparams.get('num_heads', 8),
                    dropout_rate=hyperparams.get('dropout_rate', 0.1)
                ).to(device)
            elif name == "EnsembleInside_Optuna":
                model = ModelClass(
                    X_train_scaled.shape[1], 424,
                    num_experts=hyperparams.get('num_experts', 3),
                    expert_hidden=hyperparams.get('expert_hidden', 384),
                    gate_hidden=hyperparams.get('gate_hidden', 128)
                ).to(device)
            elif name == "Wide_Default":
                model = ModelClass(X_train_scaled.shape[1], 424).to(device)
            elif name == "Attention_Default":
                model = ModelClass(X_train_scaled.shape[1], 424, hidden_size=384).to(device)
            elif name == "DeepSkip_Default":
                model = ModelClass(X_train_scaled.shape[1], 424, hidden_size=512, num_blocks=5).to(device)
            elif name == "EnsembleInside_Default":
                model = ModelClass(X_train_scaled.shape[1], 424, num_experts=4).to(device)
            else:
                model = ModelClass(X_train_scaled.shape[1], 424, hidden_size=512).to(device)
            
            # Обучение
            model, val_score = train_model_with_augmentation(
                model, X_train_t, y_train_t, X_val_t, y_val_t, 
                epochs=200, patience=20, lr=lr, model_name=name,
                use_augmentation=use_aug, augmentation_strength=aug_strength
            )
            
            models.append(model)
            model_val_scores.append(val_score)
            
            print(f"✅ {name} завершена. Best Val Score: {val_score:.4f}")
            
        except Exception as e:
            print(f"❌ ОШИБКА при обучении модели {name}: {e}")
            import traceback
            traceback.print_exc()
            continue
        
        # Очистка GPU памяти
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    # Проверка что хоть что-то обучилось
    if not models:
        raise Exception("❌ НИ ОДНА МОДЕЛЬ НЕ БЫЛА УСПЕШНО ОБУЧЕНА!")
    
    # Вычисление весов для ансамбля
    weights = np.array(model_val_scores)
    weights = weights / weights.sum()
    
    print(f"\n{'='*70}")
    print(f"ВЕСА ВСЕХ 6 МОДЕЛЕЙ С OPTUNA ОПТИМИЗАЦИЕЙ:")
    for i, (config, weight, val_score) in enumerate(zip(model_configs, weights, model_val_scores)):
        if i < len(models):
            optuna_status = "OPTUNA" if "Optuna" in config[1] else "DEFAULT"
            print(f"  Модель {i+1} ({config[1]}): вес={weight:.4f}, val_score={val_score:.4f} [{optuna_status}]")
    print(f"{'='*70}\n")
    
    is_initialized = True
    
    # Сохранение scaler и feature_cols
    joblib.dump(scaler, 'scaler_optuna.joblib')
    joblib.dump(feature_cols, 'feature_cols_optuna.joblib')
    joblib.dump(best_hyperparams, 'best_hyperparams.joblib')
    print("Scaler, feature_cols и hyperparams сохранены")

In [9]:
# Ячейка 9: Функции предсказания и оценки (ОБНОВЛЕННАЯ)
def prepare_features(df):
    global feature_cols, scaler
    
    # Создаем фичи тем же способом что и при обучении
    test_features = create_optimized_features(df, base_cols_ref=base_cols, max_features=800)
    
    # Используем только те фичи, которые были отобраны при обучении
    available_features = [f for f in feature_cols if f in test_features.columns]
    
    X_test = np.zeros((len(df), len(feature_cols)))
    
    for i, col in enumerate(feature_cols):
        if col in test_features.columns:
            X_test[:, i] = test_features[col].fillna(0).values
    
    X_test = np.nan_to_num(X_test, nan=0.0, posinf=0.0, neginf=0.0)
    
    return X_test

@torch.no_grad()
def efficient_predict_batch(test_data, models, weights=None, batch_size=1024):
    """Эффективное предсказание с батчингом"""
    if weights is None:
        weights = np.ones(len(models)) / len(models)
    
    # Если test_data уже pandas DataFrame, используем как есть
    if hasattr(test_data, 'to_pandas'):
        test_pd = test_data.to_pandas()
    else:
        test_pd = test_data
    
    X_test = prepare_features(test_pd)
    X_test_scaled = scaler.transform(X_test)
    
    # Батчинг для больших данных
    all_predictions = []
    
    for i in range(0, len(X_test_scaled), batch_size):
        batch = X_test_scaled[i:i+batch_size]
        X_batch_tensor = torch.FloatTensor(batch).to(device)
        
        batch_preds = []
        for model, weight in zip(models, weights):
            pred = model(X_batch_tensor).cpu().numpy()
            batch_preds.append(pred * weight)
        
        batch_ensemble = np.sum(batch_preds, axis=0)
        
        # Мягкое ограничение
        batch_ensemble = np.tanh(batch_ensemble * 10) * 0.1
        all_predictions.append(batch_ensemble)
    
    predictions = np.vstack(all_predictions)
    predictions = np.nan_to_num(predictions, nan=0.0, posinf=0.0, neginf=0.0)
    
    return predictions

def diagnose_predictions(predictions, targets):
    """Диагностика предсказаний для поиска проблем"""
    print("\n" + "="*70)
    print("ДИАГНОСТИКА ПРЕДСКАЗАНИЙ")
    print("="*70)
    
    print(f"\nФорма данных:")
    print(f"  Predictions: {predictions.shape}")
    print(f"  Targets: {targets.shape}")
    
    print(f"\nСтатистика predictions:")
    print(f"  Mean:   {predictions.mean():.8f}")
    print(f"  Std:    {predictions.std():.8f}")
    print(f"  Min:    {predictions.min():.8f}")
    print(f"  Max:    {predictions.max():.8f}")
    print(f"  Median: {np.median(predictions):.8f}")
    
    print(f"\nСтатистика targets:")
    print(f"  Mean:   {targets.mean():.8f}")
    print(f"  Std:    {targets.std():.8f}")
    print(f"  Min:    {targets.min():.8f}")
    print(f"  Max:    {targets.max():.8f}")
    print(f"  Median: {np.median(targets):.8f}")
    
    # Проверка на константные колонки
    const_pred_cols = []
    const_target_cols = []
    
    for i in range(predictions.shape[1]):
        if len(np.unique(predictions[:, i])) < 2:
            const_pred_cols.append(i)
        if len(np.unique(targets[:, i])) < 2:
            const_target_cols.append(i)
    
    if len(const_pred_cols) > 0:
        print(f"\n⚠️ Константные predictions колонки: {len(const_pred_cols)}/{predictions.shape[1]}")
        print(f"   Примеры индексов: {const_pred_cols[:10]}")
    else:
        print(f"\n✅ Нет константных predictions колонок")
    
    if len(const_target_cols) > 0:
        print(f"\n⚠️ Константные target колонки: {len(const_target_cols)}/{targets.shape[1]}")
        print(f"   Примеры индексов: {const_target_cols[:10]}")
    else:
        print(f"\n✅ Нет константных target колонок")
    
    # Проверка на NaN и Inf
    nan_preds = np.isnan(predictions).sum()
    inf_preds = np.isinf(predictions).sum()
    nan_targets = np.isnan(targets).sum()
    inf_targets = np.isinf(targets).sum()
    
    if nan_preds > 0 or inf_preds > 0:
        print(f"\n⚠️ Predictions: NaN={nan_preds}, Inf={inf_preds}")
    else:
        print(f"\n✅ Predictions: Нет NaN/Inf")
    
    if nan_targets > 0 or inf_targets > 0:
        print(f"\n⚠️ Targets: NaN={nan_targets}, Inf={inf_targets}")
    else:
        print(f"\n✅ Targets: Нет NaN/Inf")
    
    # Распределение знаков
    pos_preds = (predictions > 0).sum()
    neg_preds = (predictions < 0).sum()
    zero_preds = (predictions == 0).sum()
    
    pos_targets = (targets > 0).sum()
    neg_targets = (targets < 0).sum()
    zero_targets = (targets == 0).sum()
    
    total = predictions.size
    print(f"\nРаспределение знаков predictions:")
    print(f"  Positive: {pos_preds}/{total} ({100*pos_preds/total:.1f}%)")
    print(f"  Negative: {neg_preds}/{total} ({100*neg_preds/total:.1f}%)")
    print(f"  Zero:     {zero_preds}/{total} ({100*zero_preds/total:.1f}%)")
    
    print(f"\nРаспределение знаков targets:")
    print(f"  Positive: {pos_targets}/{total} ({100*pos_targets/total:.1f}%)")
    print(f"  Negative: {neg_targets}/{total} ({100*neg_targets/total:.1f}%)")
    print(f"  Zero:     {zero_targets}/{total} ({100*zero_targets/total:.1f}%)")
    
    # Проверка вариации по targets
    low_variance_preds = 0
    low_variance_targets = 0
    
    for i in range(predictions.shape[1]):
        if predictions[:, i].std() < 1e-6:
            low_variance_preds += 1
        if targets[:, i].std() < 1e-6:
            low_variance_targets += 1
    
    if low_variance_preds > 0:
        print(f"\n⚠️ Predictions с низкой дисперсией (std < 1e-6): {low_variance_preds}/{predictions.shape[1]}")
    
    if low_variance_targets > 0:
        print(f"\n⚠️ Targets с низкой дисперсией (std < 1e-6): {low_variance_targets}/{targets.shape[1]}")
    
    print("="*70 + "\n")

def calculate_kaggle_score(predictions, targets):
    """ ПРАВИЛЬНАЯ метрика соревнования: Modified Sharpe Ratio
    Score = (Mean Spearman Correlation / Std Spearman Correlation) * 100,000
    """
    print("\n" + "="*70)
    print("ВЫЧИСЛЕНИЕ KAGGLE SCORE (Modified Sharpe Ratio)")
    print("="*70)
    
    correlations = []
    failed_targets = 0
    
    # Для каждого target вычисляем Spearman correlation
    for i in range(targets.shape[1]):  # 424 targets
        pred_col = predictions[:, i]
        true_col = targets[:, i]
        
        # Проверяем что есть вариация в данных
        if len(np.unique(pred_col)) < 2 or len(np.unique(true_col)) < 2:
            failed_targets += 1
            continue
        
        # Spearman rank correlation
        try:
            corr, p_value = spearmanr(pred_col, true_col)
            if not np.isnan(corr) and not np.isinf(corr):
                correlations.append(corr)
            else:
                failed_targets += 1
        except Exception as e:
            failed_targets += 1
            continue
    
    if len(correlations) == 0:
        print("\n⚠️ КРИТИЧЕСКАЯ ОШИБКА: Не удалось вычислить ни одной корреляции!")
        print(f"   Провалено targets: {failed_targets}/{targets.shape[1]}")
        print("\n   Причины:")
        print("   - Все predictions константные для каждого target")
        print("   - Все targets константные")
        print("   - Недостаточно вариации в данных")
        
        return {
            'kaggle_score': 0.0,
            'sharpe_ratio': 0.0,
            'mean_correlation': 0.0,
            'std_correlation': 0.0,
            'median_correlation': 0.0,
            'correlations': np.array([]),
            'successful_targets': 0,
            'failed_targets': failed_targets
        }
    
    correlations = np.array(correlations)
    
    # Статистика
    mean_corr = np.mean(correlations)
    std_corr = np.std(correlations)
    median_corr = np.median(correlations)
    min_corr = np.min(correlations)
    max_corr = np.max(correlations)
    
    # Kaggle Score (масштабированный Modified Sharpe Ratio)
    if std_corr > 1e-8:  # Защита от деления на 0
        sharpe_ratio = mean_corr / std_corr
        kaggle_score = sharpe_ratio * 100000  # Масштабирование
    else:
        sharpe_ratio = 0
        kaggle_score = 0
    
    print(f"\nСтатистика Spearman Correlations:")
    print(f"  Успешно: {len(correlations)}/{targets.shape[1]} targets")
    if failed_targets > 0:
        print(f"  Провалено: {failed_targets} targets (константные значения)")
    print(f"  Mean:    {mean_corr:.6f}")
    print(f"  Median:  {median_corr:.6f}")
    print(f"  Std:     {std_corr:.6f}")
    print(f"  Min:     {min_corr:.6f}")
    print(f"  Max:     {max_corr:.6f}")
    
    # Распределение
    positive = (correlations > 0).sum()
    negative = (correlations < 0).sum()
    near_zero = (np.abs(correlations) < 0.01).sum()
    
    print(f"\n  Positive correlations: {positive}/{len(correlations)} ({100*positive/len(correlations):.1f}%)")
    print(f"  Negative correlations: {negative}/{len(correlations)} ({100*negative/len(correlations):.1f}%)")
    print(f"  Near zero (|r| < 0.01): {near_zero}/{len(correlations)} ({100*near_zero/len(correlations):.1f}%)")
    
    print(f"\n" + "="*70)
    print(f"KAGGLE SCORE (Modified Sharpe Ratio):")
    print(f"="*70)
    print(f"  Sharpe Ratio:        {sharpe_ratio:.6f}")
    print(f"  KAGGLE SCORE:        {kaggle_score:.2f}")
    
    # Интерпретация
    if kaggle_score > 100000:
        print(f"\n  ✅ ОТЛИЧНЫЙ результат! (> 100,000)")
    elif kaggle_score > 50000:
        print(f"\n  ✓ Хороший результат (> 50,000)")
    elif kaggle_score > 0:
        print(f"\n  ⚠ Слабый результат (> 0, но < 50,000)")
    else:
        print(f"\n  ❌ ПЛОХОЙ результат (отрицательный score)")
        print(f"     Модель предсказывает в противоположном направлении!")
    
    print(f"="*70 + "\n")
    
    return {
        'kaggle_score': kaggle_score,
        'sharpe_ratio': sharpe_ratio,
        'mean_correlation': mean_corr,
        'std_correlation': std_corr,
        'median_correlation': median_corr,
        'min_correlation': min_corr,
        'max_correlation': max_corr,
        'correlations': correlations,
        'successful_targets': len(correlations),
        'failed_targets': failed_targets
    }

def full_evaluation_corrected():
    """Полная оценка с ПРАВИЛЬНОЙ метрикой"""
    global models, scaler, feature_cols, base_cols, model_val_scores, is_initialized, device
    
    if not is_initialized:
        print("Модели не инициализированы!")
        return
    
    print("\n" + "🎯"*35)
    print("ПОЛНАЯ ОЦЕНКА МОДЕЛИ (ПРАВИЛЬНАЯ МЕТРИКА)")
    print("🎯"*35)
    
    # Загружаем данные
    train = pd.read_csv('/home/nicolaedrabcinski/sd_kaggle/data/raw/train.csv')
    train_labels = pd.read_csv('/home/nicolaedrabcinski/sd_kaggle/data/raw/train_labels.csv')
    
    # Создаем фичи
    train_features = create_optimized_features(train, base_cols_ref=base_cols, max_features=800)
    
    target_cols = [f'target_{i}' for i in range(424)]
    
    X_train = train_features[feature_cols].fillna(0).values
    y_train = train_labels[target_cols].fillna(0).values
    
    X_train = np.nan_to_num(X_train, nan=0.0, posinf=0.0, neginf=0.0)
    y_train = np.nan_to_num(y_train, nan=0.0, posinf=0.0, neginf=0.0)
    
    # Validation set (последние 10%)
    split_idx = int(len(X_train) * 0.9)
    X_val = X_train[split_idx:]
    y_val = y_train[split_idx:]
    
    print(f"\n1️⃣  VALIDATION SCORE:")
    print(f"Validation set size: {len(X_val)} samples")
    
    # Предсказания
    X_val_scaled = scaler.transform(X_val)
    X_val_tensor = torch.FloatTensor(X_val_scaled).to(device)
    
    all_preds = []
    with torch.no_grad():
        for model in models:
            model.eval()
            pred = model(X_val_tensor)
            pred_cpu = pred.cpu().numpy()
            all_preds.append(pred_cpu)
    
    # Взвешенный ансамбль
    weights = np.array(model_val_scores)
    weights = weights / weights.sum()
    
    val_preds = np.average(all_preds, axis=0, weights=weights)
    val_preds = np.tanh(val_preds * 10) * 0.1  # Мягкое ограничение
    val_preds = np.nan_to_num(val_preds, nan=0.0, posinf=0.0, neginf=0.0)
    
    # ДИАГНОСТИКА VALIDATION PREDICTIONS
    diagnose_predictions(val_preds, y_val)
    
    # Вычисляем ПРАВИЛЬНЫЙ Kaggle Score
    val_results = calculate_kaggle_score(val_preds, y_val)
    
    # Test set (последние 90 дней)
    print(f"\n2️⃣  TEST SCORE:")
    test_train = train.tail(90).reset_index(drop=True)
    test_labels = train_labels.tail(90).reset_index(drop=True)
    
    X_test = prepare_features(test_train)
    y_test = test_labels[target_cols].fillna(0).values
    y_test = np.nan_to_num(y_test, nan=0.0, posinf=0.0, neginf=0.0)
    
    X_test_scaled = scaler.transform(X_test)
    X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
    
    all_preds = []
    with torch.no_grad():
        for model in models:
            model.eval()
            pred = model(X_test_tensor)
            pred_cpu = pred.cpu().numpy()
            all_preds.append(pred_cpu)
    
    test_preds = np.average(all_preds, axis=0, weights=weights)
    test_preds = np.tanh(test_preds * 10) * 0.1  # Мягкое ограничение
    test_preds = np.nan_to_num(test_preds, nan=0.0, posinf=0.0, neginf=0.0)
    
    # ДИАГНОСТИКА TEST PREDICTIONS
    diagnose_predictions(test_preds, y_test)
    
    test_results = calculate_kaggle_score(test_preds, y_test)
    
    # Сравнение
    print("\n" + "="*70)
    print("📊 СРАВНЕНИЕ РЕЗУЛЬТАТОВ:")
    print("="*70)
    print(f"{'Метрика':<30} {'Validation':<20} {'Test':<20}")
    print("-"*70)
    print(f"{'KAGGLE SCORE':<30} {val_results['kaggle_score']:<20.2f} {test_results['kaggle_score']:<20.2f}")
    print(f"{'Sharpe Ratio':<30} {val_results['sharpe_ratio']:<20.6f} {test_results['sharpe_ratio']:<20.6f}")
    print(f"{'Mean Correlation':<30} {val_results['mean_correlation']:<20.6f} {test_results['mean_correlation']:<20.6f}")
    print(f"{'Std Correlation':<30} {val_results['std_correlation']:<20.6f} {test_results['std_correlation']:<20.6f}")
    print("="*70)
    
    if val_results['kaggle_score'] > 100000:
        print("✅ ОТЛИЧНЫЙ РЕЗУЛЬТАТ! Score > 100,000")
    elif val_results['kaggle_score'] > 50000:
        print("✓ Хороший результат. Score > 50,000")
    elif val_results['kaggle_score'] > 0:
        print("⚠ Слабый результат. Score > 0, но < 50,000")
    else:
        print("❌ ПЛОХОЙ результат. Отрицательный score!")
    
    print("\n" + "🎯"*35 + "\n")
    
    return {
        'validation': val_results,
        'test': test_results
    }

def create_submission_file():
    global models, scaler, feature_cols, is_initialized, device, model_val_scores
    
    print("\nСоздание submission.parquet...")
    
    if not is_initialized:
        raise Exception("Модели не инициализированы!")
    
    test = pd.read_csv('/home/nicolaedrabcinski/sd_kaggle/data/raw/test.csv')
    
    print("Генерация предсказаний...")
    
    # Вычисляем веса
    weights = np.array(model_val_scores)
    weights = weights / weights.sum()
    
    # Используем батчинговое предсказание
    predictions = efficient_predict_batch(test, models, weights)
    
    submission = pd.DataFrame({'date_id': test['date_id'].values})
    for i in range(424):
        submission[f'target_{i}'] = predictions[:, i]
    
    if 'is_scored' in test.columns:
        submission = submission[test['is_scored'] == True].reset_index(drop=True)
    
    submission = submission.fillna(0).replace([np.inf, -np.inf], 0)
    submission.to_parquet('submission_optuna.parquet', index=False, engine='pyarrow')
    
    print(f"Готово: {submission.shape}")
    
    # Очищаем GPU память
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

In [10]:
# Ячейка 10: Визуализация Optuna и основной пайплайн
def visualize_optuna_results():
    """Визуализация результатов Optuna с сохранением в PNG"""
    if not optuna_studies:
        print("Нет данных Optuna для визуализации")
        return
    
    # Создаем папку для визуализаций
    import os
    os.makedirs('optuna_visualizations', exist_ok=True)
    
    for model_type, study in optuna_studies.items():
        print(f"\n📊 ВИЗУАЛИЗАЦИЯ РЕЗУЛЬТАТОВ OPTUNA ДЛЯ {model_type}:")
        
        # Лучшие параметры
        print(f"Лучшие параметры: {study.best_trial.params}")
        print(f"Лучший score: {study.best_value:.4f}")
        
        try:
            # Важность параметров
            fig = optuna.visualization.plot_param_importances(study)
            fig.update_layout(title=f"Важность параметров - {model_type}")
            fig.write_image(f"optuna_visualizations/{model_type}_param_importance.png")
            # fig.show()
            print(f"✅ Сохранен: optuna_visualizations/{model_type}_param_importance.png")
        except Exception as e:
            print(f"Не удалось создать график важности параметров: {e}")
        
        try:
            # Параллельные координаты
            fig = optuna.visualization.plot_parallel_coordinate(study)
            fig.update_layout(title=f"Параллельные координаты - {model_type}")
            fig.write_image(f"optuna_visualizations/{model_type}_parallel_coord.png")
            # fig.show()
            print(f"✅ Сохранен: optuna_visualizations/{model_type}_parallel_coord.png")
        except Exception as e:
            print(f"Не удалось создать график параллельных координат: {e}")
        
        try:
            # История оптимизации
            fig = optuna.visualization.plot_optimization_history(study)
            fig.update_layout(title=f"История оптимизации - {model_type}")
            fig.write_image(f"optuna_visualizations/{model_type}_optimization_history.png")
            # fig.show()
            print(f"✅ Сохранен: optuna_visualizations/{model_type}_optimization_history.png")
        except Exception as e:
            print(f"Не удалось создать график истории оптимизации: {e}")
        
        try:
            # Распределение параметров
            fig = optuna.visualization.plot_slice(study)
            fig.update_layout(title=f"Срез параметров - {model_type}")
            fig.write_image(f"optuna_visualizations/{model_type}_slice_plot.png")
            # fig.show()
            print(f"✅ Сохранен: optuna_visualizations/{model_type}_slice_plot.png")
        except Exception as e:
            print(f"Не удалось создать график среза параметров: {e}")
    
    print(f"\n🎉 Все визуализации сохранены в папку 'optuna_visualizations/'")

In [11]:
def main_with_optuna(use_optuna=True, optuna_trials=30, run_evaluation=True):
    global is_initialized
    
    print("ЗАПУСК ПАЙПЛАЙНА С OPTUNA ОПТИМИЗАЦИЕЙ")
    print("="*60)
    
    if not is_initialized:
        print("Начало обучения с оптимизацией гиперпараметров...")
        initialize_models_with_optuna(use_optuna=use_optuna, optuna_trials=optuna_trials)
    
    # Визуализация результатов Optuna
    if use_optuna:
        visualize_optuna_results()
    
    # Полная оценка модели
    if run_evaluation:
        print("\nЗапуск полной оценки модели...")
        results = full_evaluation_corrected()
    
    # Создание submission
    print("\nСоздание submission файла...")
    create_submission_file()
    
    print("\n✅ OPTUNA АНСАМБЛЬ ГОТОВ!")
    print(f"🔢 Количество моделей: {len(models)}")
    print(f"🎯 Использована оптимизация гиперпараметров: {use_optuna}")
    if use_optuna:
        print(f"🔬 Количество trials на модель: {optuna_trials}")
    if run_evaluation:
        print(f"🏆 Ожидаемый Kaggle Score: {results['validation']['kaggle_score']:.2f}")
    print(f"📁 Submission файл: submission_optuna.parquet")
    print(f"\n🚀 ОТПРАВЛЯЙТЕ НА KAGGLE!")

# Запуск пайплайна
if __name__ == "__main__":
    main_with_optuna(use_optuna=True, optuna_trials=10, run_evaluation=True)

ЗАПУСК ПАЙПЛАЙНА С OPTUNA ОПТИМИЗАЦИЕЙ
Начало обучения с оптимизацией гиперпараметров...
ИНИЦИАЛИЗАЦИЯ ВСЕХ 6 МОДЕЛЕЙ С OPTUNA ОПТИМИЗАЦИЕЙ НА cuda


Базовых колонок: 557
Создание оптимизированных признаков...
Обрабатывается 100 базовых колонок...
Достигнут лимит в 800 признаков
Создано 1507 признаков после фильтрации
Векторизованный отбор признаков с GPU...


[I 2025-10-14 21:35:36,469] A new study created in memory with name: no-name-c0673959-413c-4696-880c-8a8fdc509186


Отобрано 500 лучших признаков
Лучший признак: US_Stock_EMB_adj_open_volatility_20 (корреляция: 0.0607)
Train: 1666, Validation: 295

OPTUNA ОПТИМИЗАЦИЯ ДЛЯ SimpleEffective
Количество trials: 10


  0%|          | 0/10 [00:00<?, ?it/s]

...

[I 2025-10-14 22:09:19,049] Trial 0 finished with value: 0.005361454339121452 and parameters: {'hidden_size': 512, 'num_heads': 4, 'dropout_rate': 0.18276176448069797, 'lr': 0.00011519503227045127, 'augmentation_strength': 0.5452673022815104}. Best is trial 0 with value: 0.005361454339121452.
Ошибка в trial 1: 
[I 2025-10-14 22:10:33,232] Trial 1 finished with value: 0.0 and parameters: {'hidden_size': 256, 'num_heads': 4, 'dropout_rate': 0.27973626790277245, 'lr': 5.583958397756887e-05, 'augmentation_strength': 0.42824279979848073}. Best is trial 0 with value: 0.005361454339121452.
Ошибка в trial 2: 
[I 2025-10-14 22:11:48,021] Trial 2 finished with value: 0.0 and parameters: {'hidden_size': 256, 'num_heads': 4, 'dropout_rate': 0.2987334861149724, 'lr': 3.6354708934343e-05, 'augmentation_strength': 0.5502249506609751}. Best is trial 0 with value: 0.005361454339121452.
[I 2025-10-14 22:13:10,550] Trial 3 finished with value: -0.00028358890992626295 and parameters: {'hidden_size': 256, 

[I 2025-10-14 22:19:25,238] A new study created in memory with name: no-name-19ac556e-8fad-4402-abaa-d88bc6c3c62c


Ошибка в trial 9: 
[I 2025-10-14 22:19:25,236] Trial 9 finished with value: 0.0 and parameters: {'hidden_size': 512, 'num_heads': 4, 'dropout_rate': 0.14732108980066821, 'lr': 1.1021711463123248e-05, 'augmentation_strength': 0.5065639840918428}. Best is trial 8 with value: 0.009824960685171762.

ЛУЧШИЕ ГИПЕРПАРАМЕТРЫ ДЛЯ Attention:
  hidden_size: 1024
  num_heads: 4
  dropout_rate: 0.18413713429117845
  lr: 4.710208379346669e-05
  augmentation_strength: 0.6852492269167936
Лучший val_score: 0.0098
✅ Attention оптимизирована. Best score: 0.0098

OPTUNA ОПТИМИЗАЦИЯ ДЛЯ EnsembleInside
Количество trials: 10


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2025-10-14 22:20:48,390] Trial 0 finished with value: 0.0037545842268386694 and parameters: {'num_experts': 3, 'expert_hidden': 512, 'gate_hidden': 128, 'lr': 0.00013319545921315043, 'augmentation_strength': 0.5855723596499758}. Best is trial 0 with value: 0.0037545842268386694.
Ошибка в trial 1: 
[I 2025-10-14 22:20:57,074] Trial 1 finished with value: 0.0 and parameters: {'num_experts': 3, 'expert_hidden': 384, 'gate_hidden': 128, 'lr': 0.004449354822094503, 'augmentation_strength': 0.7065910235735845}. Best is trial 0 with value: 0.0037545842268386694.
[I 2025-10-14 22:21:59,507] Trial 2 finished with value: 0.004156865878527825 and parameters: {'num_experts': 5, 'expert_hidden': 384, 'gate_hidden': 128, 'lr': 0.009789164377206169, 'augmentation_strength': 0.586605612296109}. Best is trial 2 with value: 0.004156865878527825.
[I 2025-10-14 22:23:21,754] Trial 3 finished with value: 0.006792392885787304 and parameters: {'num_experts': 5, 'expert_hidden': 384, 'gate_hidden': 128, 'l